# LangChain Tools and Agents Exploration
Learn how to create and use LangChain tools, build agents that can choose between multiple tools, and add memory to maintain conversation context.

## Imports and Environment Setup

In [37]:
from langchain.agents import initialize_agent, tools, load_tools
from langchain.agents.agent_types import AgentType
from langchain.memory import ConversationBufferMemory
from langchain_groq import ChatGroq
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_experimental.tools import PythonREPLTool
from langchain.tools import tool, Tool

import os
from dotenv import load_dotenv

load_dotenv()

True

## Setting Up Groq LLM and Memory

In [38]:
# Setting up LLM
llm = ChatGroq(
    groq_api_key=groq_api_key,
    model_name="llama3-70b-8192",
    temperature=0
)

# Setting up Memory
memory = ConversationBufferMemory(
    memory_key="chat_history", # Key to match it with the agent
    return_messages=True # To see full message objects, not just text
)

## Setting Up Tools
1. DuckDuckGoSearch
2. PythonREPL
3. Calculator

In [39]:
# Calculator Tool
def calculate(expression: str) -> str:
    """Calculate mathematical expressions safely"""
    try:
        # Replace common math expressions
        expression = expression.replace("x", "*").replace("X", "*")
        result = eval(expression)
        return f"The result is: {result}"
    except Exception as e:
        return f"Error in calculation: {str(e)}"

calculator_tool = Tool(
    name="Calculator",
    description="Useful for mathematical calculations. Input should be a mathematical expression like '2+2' or '107000000*0.1'",
    func=calculate
)

# DuckDuckGo Search Tool
search_tool = DuckDuckGoSearchRun()
print(f"Search tool loaded: {search_tool.name}")

# Python REPL Tool
python_tool = PythonREPLTool()
print(f"Python REPL tool loaded: {python_tool.name}")



Search tool loaded: duckduckgo_search
Python REPL tool loaded: Python_REPL


In [40]:
# All tools 
tools =  [calculator_tool, search_tool, python_tool]


print("All tools loaded successfully:\n----------------------------------")
for i, tool in enumerate(tools, 1):
    print(f"{i}. {tool.name}\nDescription: {tool.description}\n")
    print()

All tools loaded successfully:
----------------------------------
1. Calculator
Description: Useful for mathematical calculations. Input should be a mathematical expression like '2+2' or '107000000*0.1'


2. duckduckgo_search
Description: A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.


3. Python_REPL
Description: A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.




## Initialzing the Agent

In [41]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent= AgentType.ZERO_SHOT_REACT_DESCRIPTION    ,
    memory=memory,
    verbose=True
)

## Testing the Agent

In [42]:
response = agent.run("What is the square root of 256? Then tell me who is the president of France.")
print(response)



> Entering new AgentExecutor chain...
Thought: I need to calculate the square root of 256 and then find out who is the current president of France.

Action: Calculator
Action Input: "256**0.5"
Observation: The result is: 16.0
Thought:I have the square root of 256, which is 16.0. Now I need to find out who is the current president of France.

Action: duckduckgo_search
Action Input: "who is the president of France"

d:\Virtual Enviroments\langchain-rag-env\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:



Observation: chairman, president ， CEO的区别1、职位名称不同。chairman的职位名称翻译成汉语为董事长。president的职位名称翻译成汉族为总经理或总裁。ceo为Chief Executive Officer简称，职位名称翻译成汉语为首席执行官。2 Jul 24, 2024 · VP是什么职务VP，全称为Vice President，是企业中一种重要的高级管理职务。它位于总经理 (General Manager)之下，通常分为第一副总裁 (FVP)、副总裁 (AVP)助理等不同层级。VP的主要职责包括协助CEO，即首席执行官，执行 如果说CEO是总理，那么President就是掌握日常工作的第一副总理，如果这两个职位不属于同一人，那么CEO的地位稍微高一点。 我没有调查CEO和President最早是什么时候分开的，但最早的著名例子发生在1960年代的福特汽车公司。 "Chairman" 和 "President" 都是职位头衔，但 "Chairman" 主要用于组织、公司、委员会等的领导者，其职责是主持和指导会议；而 "President" 则更广泛地用于组织、公司、学术机构或国家的领导者，其职责可能包括监督日常运营、制定政策等，特别在政府体系中，"President" 是某些国家的国家元首。 大家可以先 ... 此外，有些学校的校长还有另外的称呼：President & Vice-Chancellor或Rector & Vice-Chancellor。 诺丁汉大学的现任Vice Chancellor and President是Shearer West，这位女教授从2017年任执行校长至今，她同时也是诺丁汉马来西亚和宁波分校（宁波大学诺丁汉大学）的校长。
Thought:Thought: I have the observation from the duckduckgo_search action, but it seems to be a mix of information about job titles and their differences. I need to extract the relevant information about the current president of Franc

d:\Virtual Enviroments\langchain-rag-env\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:



Observation: Emmanuel Jean-Michel Frédéric Macron[b] (born 21 December 1977) is a French politician who has served as President of France and … Mar 27, 2025 · Find out all the news of the President Emmanuel Macron on the official website and discover our pages on the … 6 days ago · Emmanuel Macron is a French banker and politician who was elected president of France in 2017. Macron was the … Emmanuel Jean-Michel Frédéric Macron CBE (French: [emanɥɛl makʁɔ̃]; born 21 December 1977 in Amiens) is a French politician, … Sep 19, 2024 · Find out who the current president of France is, his political career, his actions, and his impact on the country. Stay …
Thought:I have the relevant information about the current president of France.

Final Answer: The square root of 256 is 16.0, and the current president of France is Emmanuel Macron.

> Finished chain.
The square root of 256 is 16.0, and the current president of France is Emmanuel Macron.


## Testing Each Tool Individually

### 1) Calculator

In [43]:
response1 = agent.run("What is the square root of 144?")
print(response1)



> Entering new AgentExecutor chain...
Thought: I need to calculate the square root of 144.

Action: Calculator
Action Input: "144**0.5"
Observation: The result is: 12.0
Thought:I now know the final answer
Final Answer: 12.0

> Finished chain.
12.0


### 2) DuckDuckGoSearch

In [44]:
response2 = agent.run("What is the capital of France?")
print(response2)



> Entering new AgentExecutor chain...
Thought: I need to find the capital of France, which is a piece of general knowledge. I can use DuckDuckGo search to find the answer.

Action: duckduckgo_search
Action Input: "capital of France"

d:\Virtual Enviroments\langchain-rag-env\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:



Observation: Nov 26, 2024 · Answer: Explanation: The capital of France is Paris, and there are no other official capitals. However, there are several other major cities in France that are important, such as: … Dec 19, 2024 · París [pa.'ɾis] (capital of France) Paris ['pa.ɾis] (Hector's brother in The Iliad) *parís [pa.'ɾis] you (familiar plural / classic Spanish "vos" formal singular) give birth: vosotras parís ... May 6, 2020 answer of Capital Of France clue in NYT Crossword Puzzle. There is One Answer total, Euros is the most recent and it has 5 letters. Dec 27, 2022 · Answer: Paris is the capital of France, the largest country of Europe with 550 000 km2 (65 millions inhabitants). Paris has 2.234 million inhabitants end 2011. She is the core of … Aug 30, 2023 · The Letter 'N' : London France ^ ^ ------->^The words pants and France do not rhyme because there is a t sound in pants which is not in France. If the word were Frants, it …
Thought:Thought: I now know the final answer
Final 

### 3) Python REPL Tool

In [45]:
response3 = response3 = agent.run("Write Python code to create a list of the first 5 prime numbers and show the result")
print(response3)



> Entering new AgentExecutor chain...
Thought: I need to write a Python function to generate prime numbers and then use it to create a list of the first 5 prime numbers.

Action: Python_REPL
Action Input: `def is_prime(n): return n > 1 and all(n%i != 0 for i in range(2, int(n**0.5) + 1))`

Observation: 
Thought:Thought: Now I have a function to check if a number is prime, I need to use it to generate the first 5 prime numbers.

Action: Python_REPL
Action Input: `primes = []; i = 2; while len(primes) < 5: if is_prime(i): primes.append(i); i += 1; print(primes)`

Observation: SyntaxError('invalid syntax', ('<string>', 1, 21, 'primes = []; i = 2; while len(primes) < 5: if is_prime(i): primes.append(i); i += 1; print(primes)\n', 1, 26))
Thought:It looks like I made a syntax error!

Thought: I need to fix the syntax error in my Python code.

Action: Python_REPL
Action Input: `primes = []; i = 2; while len(primes) < 5: if is_prime(i): primes.append(i); i += 1; print(primes)` (with proper i

## Testing Multiple Tools

In [46]:
response4 = agent.run("Find the population of Egypt, then calculate what 10% of that population would be")
print(response4)



> Entering new AgentExecutor chain...
Thought: I need to find the current population of Egypt, and then calculate 10% of that number.

Action: duckduckgo_search
Action Input: "population of Egypt"

d:\Virtual Enviroments\langchain-rag-env\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:



Observation: Population of Egypt: current, historical, and projected population, growth rate, immigration, median age, total fertility rate (TFR), population density, urbanization ... Egypt Population History From 1970 to 2010, there was a population boom due to advancements in agricultural productivity and medical fields, which were made possible due to the Green … 2010 population density Egypt is the most populous country in the Middle East, and the third-most populous on the African continent, after Nigeria, Ethiopia. [3][4] About 95% [5] of the country's … Population Pyramid, Age Structure, Sex Ratio (Males to Females), Life Expectancy, Dependency Ratio of Egypt The current population of Egypt is 118,520,786 as of Monday, August 4, 2025, based on Worldometer's elaboration of the latest United Nations data 1. Egypt 2025 population is …
Thought:Thought: I now have the current population of Egypt, which is 118,520,786. I need to calculate 10% of this number.

Action: Calculator
Actio

In [47]:
response5 = agent.run("Use Python to calculate the factorial of 8, then tell me if that number is greater than 40000")
print(f"Agent Response: {response5}")



> Entering new AgentExecutor chain...
Thought: I need to calculate the factorial of 8 using Python.

Action: Python_REPL
Action Input: `import math; print(math.factorial(8))`
Observation: 40320

Thought:Thought: Now that I have the factorial of 8, I need to compare it to 40000.

Action: Calculator
Action Input: '40320 > 40000'
Observation: The result is: 40320 > 40000
Thought:I now know the final answer
Final Answer: Yes, the factorial of 8 is greater than 40000.

> Finished chain.
Agent Response: Yes, the factorial of 8 is greater than 40000.


In [49]:
response6 = agent.run("Search for the current exchange rate of USD to Egyptian Pound, then write Python code to convert 1000 USD to EGP")
print(f"Agent Response: {response6}")



> Entering new AgentExecutor chain...
Thought: I need to find the current exchange rate of USD to EGP, and then write Python code to convert 1000 USD to EGP.

Action: duckduckgo_search
Action Input: "USD to EGP exchange rate"

d:\Virtual Enviroments\langchain-rag-env\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:



Observation: No good DuckDuckGo Search Result was found
Thought:It seems like DuckDuckGo search didn't yield a good result. Let me try again.

Action: duckduckgo_search
Action Input: "current USD to EGP exchange rate"

d:\Virtual Enviroments\langchain-rag-env\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:



Observation: May 8, 2025 · CBE Exchange RatesCBE Official Exchange rates and prices are expressed in pounds. Rates for Date: … 23 rows · 23 hours ago · Check the live USD to EGP exchange rate today in all Egyptian banks. Follow real-time price … 1 day ago · Get the latest and best $1 US Dollar to Egyptian Pounds rate for FREE. USD/EGP - Live exchange rates, banks, historical data & currency charts. 1 day ago · Learn the value of 1 United States Dollar (USD) in Egyptian Pounds (EGP) today. The dynamics of the … 5 days ago · Current exchange rate US DOLLAR (USD) to EGYPTIAN POUND (EGP) including currency converter, …
Thought:It seems like I got a list of search results from DuckDuckGo. Let me try to find the current exchange rate from one of these results.

Action: duckduckgo_search
Action Input: "USD to EGP exchange rate today"

d:\Virtual Enviroments\langchain-rag-env\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:



Observation: May 8, 2025 · CBE Official Exchange rates and prices are expressed in pounds. This page was last updated 04 Aug 2025. 23 rows · 23 hours ago · Check the live USD to EGP exchange rate today in all Egyptian banks. Follow real-time price updates for the US Dollar to Egyptian Pound and stay ahead of market … 1 day ago · The cost of 1 United States Dollar in Egyptian Pounds today is £48.45 according to the “Open Exchange Rates”, compared to yesterday, the exchange rate increased by 0.02% (by … 5 days ago · Current exchange rate US DOLLAR (USD) to EGYPTIAN POUND (EGP) including currency converter, buying & selling rate and historical conversion chart. 40 rows · 1 day ago · This is the live USD EGP rate forex data page, displaying the FX price for the USD/EGP. The FX rate self-updates every few seconds. Compare exchange rates with base …
Thought:It seems like I finally found the current exchange rate: 1 USD is approximately 48.45 EGP.

Action: Python_REPL
Action Input: `exchange